In [37]:
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
import numpy as np
from math import pi, cos , sin ,exp ,ceil,isnan, isinf
from IPython import display

In [38]:
def psi(r):
    q = (r+abs(r))/(1+abs(r))
    if isinf(q) or isnan(q):
        return 0
    else: return q

In [39]:
def godunov(ul,ur):
    fl = 0.5*ul**2
    fr = 0.5*ur**2
    if ul*ur<0 and ul<ur:
        f = 0
    elif ul>=ur:
        f = max(fl,fr)
    elif ul<ur:
        f = min(fl,fr)
    return f

In [40]:
N = 60  #number of cells
cfl = 0.1
endtime = 4*pi #simulation time
initials = '.5' #  'sin' or '.5' 

In [41]:
h = 2*pi/N
delT = cfl*h 
t = np.arange(0,endtime+delT,delT)
x = np.arange(h/2,2*pi,h)
frames = len(t)
S = np.zeros([frames,N])

In [42]:
if initials == 'sin':
    S[0,:] = np.sin(x)
elif initials == '.5':
    S[0,:] = np.sin(x)+0.5

In [43]:
def derivate(s):
    ar = np. concatenate((s[N-2],s[N-1],s,s[0],s[1]),axis=None)
    rr = np.zeros(N+1)
    rl = np.zeros(N+1)
    psisr =  np.zeros(N+1)
    psisl =  np.zeros(N+1)
    ur =  np.zeros(N+1)
    ul =  np.zeros(N+1)
    for i in range(N+1):
        rl[i] = (ar[i+2]-ar[i+1])/(ar[i+1]-ar[i])
        rr[i] = (ar[i+1]-ar[i+2])/(ar[i+2]-ar[i+3])
        psisl[i] = psi(rl[i])
        psisr[i] = psi(rr[i])
    for i in range(N+1):
        ul[i] = ar[i+1]+0.5 * psisl[i]*(ar[i+1]-ar[i])
        ur[i] = ar[i+2]+0.5 * psisr[i]*(ar[i+2]-ar[i+3])
    dudt = np.zeros(N)
    for i in range(N):
        dudt[i] = (-godunov(ul[i+1],ur[i+1])+godunov(ul[i],ur[i]))/(h)        
    return dudt
    

In [44]:
for i in range(frames-1):
    k1 = derivate(S[i,:])
    u2_1 = S[i,:] +0.5*delT*k1
    k2 = derivate(u2_1)
    u2_2 = S[i,:] +0.5*delT*k2
    k3 = derivate(u2_2)
    u3 = S[i,:] +delT*k3
    k4 = derivate(u3)
    S[i+1,:] = S[i,:] + (1/6)*delT*(k1 +2*k2+2*k3+k4)


C:\Users\Rayan\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Rayan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\Rayan\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [46]:
def animate(i):
    y = S[i,:]
    line.set_data((x,y))

fig = plt.figure()
lines = plt.plot([])
line = lines[0]
plt.xlim(0, 2*pi)
plt.ylim(-1.7,1.7)

anim = FuncAnimation(fig,animate,frames = frames, interval = 30)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()